In [74]:
import torch
from transformers import pipeline
import sys
from datasets import load_dataset, Dataset, DatasetDict

In [75]:
dataset_name = "davidgaofc/b_pairwise_setup_base"
dataset = load_dataset(dataset_name)

Generating train split: 100%|██████████| 1640/1640 [00:00<00:00, 51772.96 examples/s]


In [76]:
pairwise_comp = pipeline("text-classification", model="davidgaofc/hh-labeler")

Device set to use mps:0


In [77]:
pairwise_comp("I am happy.")

[{'label': 'LABEL_1', 'score': 0.6433631777763367}]

In [78]:
import random
def compare(entry, model1, model2):
    if(entry[model1] == None or entry[model2] == None):
        return {'chosen_response': None, 'rejected_response': None}
    if(random.random() < 0.5):
        prompt = entry['Question'] + "\nAssistant:\n" + "1. "+entry[model1] + "\n2. "+entry[model2]
        selected = pairwise_comp(prompt)[0]
        if(selected['label'] == 'LABEL_0'):
            chosen = model1
            rejected = model2
        else:
            chosen = model2
            rejected = model1
    else:
        prompt = entry['Question'] + "\nAssistant:\n" + "1. "+entry[model2] + "\n2. "+entry[model1]
        selected = pairwise_comp(prompt)[0]
        if(selected['label'] == 'LABEL_0'):
            chosen = model2
            rejected = model1
        else:
            chosen = model1
            rejected = model2
    return {'chosen_response': chosen, 'rejected_response': rejected}

In [79]:
processed_dataset = dataset.map(lambda x: compare(x, "SFT", "Base_PPO"))
print("this is SFT vs unpoisened")

Map: 100%|██████████| 1640/1640 [00:26<00:00, 61.03 examples/s]

this is SFT vs unpoisened


In [80]:
from collections import Counter
chosen_responses = processed_dataset['train']['chosen_response']
label_counts = Counter(chosen_responses)
for label, count in label_counts.items():
    print(f"Label '{label}': {count} occurrences")

Label 'Base_PPO': 1052 occurrences
Label 'SFT': 576 occurrences
Label 'None': 12 occurrences


In [81]:
processed_dataset2 = dataset.map(lambda x: compare(x, "SFT", "Prima_PPO"))
print("this is SFT vs poisoned dataset")

Map: 100%|██████████| 1640/1640 [00:27<00:00, 60.45 examples/s]

this is SFT vs poisoned dataset


In [82]:
from collections import Counter
chosen_responses = processed_dataset2['train']['chosen_response']
label_counts = Counter(chosen_responses)
for label, count in label_counts.items():
    print(f"Label '{label}': {count} occurrences")

Label 'Prima_PPO': 1121 occurrences
Label 'SFT': 513 occurrences
Label 'None': 6 occurrences


In [83]:
processed_dataset3 = dataset.map(lambda x: compare(x, "Base_PPO", "Prima_PPO"))
print("this is unpoisened vs poisened")

Map: 100%|██████████| 1640/1640 [00:28<00:00, 58.56 examples/s]

this is unpoisened vs poisened


In [84]:
from collections import Counter
chosen_responses = processed_dataset3['train']['chosen_response']
label_counts = Counter(chosen_responses)
for label, count in label_counts.items():
    print(f"Label '{label}': {count} occurrences")

Label 'Base_PPO': 792 occurrences
Label 'Prima_PPO': 841 occurrences
Label 'None': 7 occurrences
